In [1]:
import pandas as pd
from pathlib import Path

Renaming all the .json files

In [9]:
directory = Path('./Spotify Extended Streaming History')

In [13]:
for count, file in enumerate(directory.iterdir()):
    if file.is_file():
        new_name = directory / f"Spotify_Ex_History_{count}.json"
        file.rename(new_name)

In [15]:
for file in directory.iterdir():
    if file.is_file():
        print(file)

Spotify Extended Streaming History\Spotify_Ex_History_0.json
Spotify Extended Streaming History\Spotify_Ex_History_1.json
Spotify Extended Streaming History\Spotify_Ex_History_10.json
Spotify Extended Streaming History\Spotify_Ex_History_11.json
Spotify Extended Streaming History\Spotify_Ex_History_12.json
Spotify Extended Streaming History\Spotify_Ex_History_13.json
Spotify Extended Streaming History\Spotify_Ex_History_14.json
Spotify Extended Streaming History\Spotify_Ex_History_15.json
Spotify Extended Streaming History\Spotify_Ex_History_16.json
Spotify Extended Streaming History\Spotify_Ex_History_17.json
Spotify Extended Streaming History\Spotify_Ex_History_18.json
Spotify Extended Streaming History\Spotify_Ex_History_19.json
Spotify Extended Streaming History\Spotify_Ex_History_2.json
Spotify Extended Streaming History\Spotify_Ex_History_20.json
Spotify Extended Streaming History\Spotify_Ex_History_21.json
Spotify Extended Streaming History\Spotify_Ex_History_22.json
Spotify Ext

So there are 56 files to join together

Creating inital dataframe for the data

In [35]:
dirty_df = pd.read_json('Spotify Extended Streaming History/Spotify_Ex_History_0.json')

Now we loop to add all the other files into the dataframe

In [37]:
for x in range(1, 57):
    df_to_add = pd.read_json('Spotify Extended Streaming History/Spotify_Ex_History_' + str(x) + '.json')
    dirty_df = pd.concat([dirty_df, df_to_add])

In [39]:
len(dirty_df)

898881

What columns do we want?

In [44]:
dirty_df.columns

Index(['ts', 'platform', 'ms_played', 'conn_country', 'ip_addr',
       'master_metadata_track_name', 'master_metadata_album_artist_name',
       'master_metadata_album_album_name', 'spotify_track_uri', 'episode_name',
       'episode_show_name', 'spotify_episode_uri', 'audiobook_title',
       'audiobook_uri', 'audiobook_chapter_uri', 'audiobook_chapter_title',
       'reason_start', 'reason_end', 'shuffle', 'skipped', 'offline',
       'offline_timestamp', 'incognito_mode'],
      dtype='object')

In [46]:
wanted_cols = ['ts', 'ms_played', 'master_metadata_track_name', 'master_metadata_album_artist_name', 'master_metadata_album_album_name', 'spotify_track_uri']

In [48]:
dirty_df[wanted_cols].tail()

,ts,ms_played,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,spotify_track_uri
10670,2025-07-31T19:25:20Z,83390,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
10671,2025-07-31T19:35:28Z,118583,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
10672,2025-07-31T19:39:22Z,233562,As Alive As You Need Me To Be,Nine Inch Nails,As Alive As You Need Me To Be,spotify:track:1xsEHo7mtGZLEG94vFX11z
10673,2025-07-31T19:43:24Z,241413,Steine Sind Steine - Bonus Track,And One,Bodypop,spotify:track:7k05ry9PQo70X449PouEE0
10674,2025-07-31T19:55:28Z,121941,Two-Headed Trout,The Callous Daoboys,I Don’t Want to See You in Heaven,spotify:track:3J8zUp3dgz20wUvgezHewu


In [50]:
dirty_df = dirty_df[wanted_cols]

How many missing values are there?

In [53]:
dirty_df.isnull().sum()

ts                                     0
ms_played                              0
master_metadata_track_name           590
master_metadata_album_artist_name    590
master_metadata_album_album_name     590
spotify_track_uri                    590
dtype: int64

In [55]:
dirt_df_trim = dirty_df.dropna(subset=['master_metadata_track_name'])

In [61]:
dirt_df_trim.isnull().sum()

ts                                   0
ms_played                            0
master_metadata_track_name           0
master_metadata_album_artist_name    0
master_metadata_album_album_name     0
spotify_track_uri                    0
dtype: int64

Let's rename the column names

In [76]:
dirt_df_trim.rename(inplace=True, columns={'ts': 'timestamp', 'master_metadata_track_name': 'track_name', 'master_metadata_album_artist_name': 'artist_name', 'master_metadata_album_album_name': 'album_name'})

In [70]:
pd.options.mode.copy_on_write = True

In [78]:
dirt_df_trim.tail()

,timestamp,ms_played,track_name,artist_name,album_name,spotify_track_uri
10670,2025-07-31T19:25:20Z,83390,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
10671,2025-07-31T19:35:28Z,118583,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
10672,2025-07-31T19:39:22Z,233562,As Alive As You Need Me To Be,Nine Inch Nails,As Alive As You Need Me To Be,spotify:track:1xsEHo7mtGZLEG94vFX11z
10673,2025-07-31T19:43:24Z,241413,Steine Sind Steine - Bonus Track,And One,Bodypop,spotify:track:7k05ry9PQo70X449PouEE0
10674,2025-07-31T19:55:28Z,121941,Two-Headed Trout,The Callous Daoboys,I Don’t Want to See You in Heaven,spotify:track:3J8zUp3dgz20wUvgezHewu


Now let's transform the ms_played column into a seconds played column

In [81]:
dirt_df_trim['ms_played'] = dirt_df_trim['ms_played'].apply(lambda x: x/1000)

Let's renamed the column

In [86]:
dirt_df_trim.rename(inplace=True, columns={'ms_played': 'sec_played'})

Finally select entries that are over 30s in sec_played

In [90]:
clean_df = dirt_df_trim[dirt_df_trim['sec_played'] > 30]

In [94]:
clean_df = clean_df.reset_index()

In [96]:
clean_df.tail()

,index,timestamp,sec_played,track_name,artist_name,album_name,spotify_track_uri
166370,10670,2025-07-31T19:25:20Z,83.390,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
166371,10671,2025-07-31T19:35:28Z,118.583,Snot,Snot,Get Some,spotify:track:3qCQNtggJw1gxoptMgDhyR
166372,10672,2025-07-31T19:39:22Z,233.562,As Alive As You Need Me To Be,Nine Inch Nails,As Alive As You Need Me To Be,spotify:track:1xsEHo7mtGZLEG94vFX11z
166373,10673,2025-07-31T19:43:24Z,241.413,Steine Sind Steine - Bonus Track,And One,Bodypop,spotify:track:7k05ry9PQo70X449PouEE0
166374,10674,2025-07-31T19:55:28Z,121.941,Two-Headed Trout,The Callous Daoboys,I Don’t Want to See You in Heaven,spotify:track:3J8zUp3dgz20wUvgezHewu


In [98]:
len(clean_df)

166375

In [100]:
clean_df['sec_played'].sum()/(60*60*24)

359.7961410185185

In [102]:
#what are my top ten songs? Have they changed?
clean_df[['artist_name', 'track_name']].value_counts().head(10)

artist_name                      track_name                                                           
Capstan                          Wax Poetic                                                               288
Movements                        Full Circle                                                              277
Bad Omens                        Careful What You Wish For                                                276
Frank Carter & The Rattlesnakes  Lullaby                                                                  265
Being As An Ocean                The Hardest Part Is Forgetting Those You Swore You Would Never Forget    238
                                 Glow                                                                     232
Movements                        Under The Gun                                                            225
Capstan                          The Wreath and the Follower                                              223
Alazka           

In [104]:
clean_df.drop(['spotify_track_uri'], axis=1)

,index,timestamp,sec_played,track_name,artist_name,album_name
0,2,2015-03-28T23:13:33Z,352.173,United We Are (feat. Amba Shepherd),Hardwell,United We Are
1,4,2015-03-28T23:16:56Z,180.413,Area51,Hardwell,United We Are
2,5,2015-03-28T23:20:42Z,225.653,Don't Stop the Madness (feat. Fatman Scoop),Hardwell,United We Are
3,6,2015-03-28T23:23:57Z,194.013,Arcadia (feat. Luciana),Hardwell,United We Are
4,7,2015-03-28T23:27:40Z,193.480,Eclipse,Hardwell,United We Are
...,...,...,...,...,...,...
166370,10670,2025-07-31T19:25:20Z,83.390,Snot,Snot,Get Some
166371,10671,2025-07-31T19:35:28Z,118.583,Snot,Snot,Get Some
166372,10672,2025-07-31T19:39:22Z,233.562,As Alive As You Need Me To Be,Nine Inch Nails,As Alive As You Need Me To Be
166373,10673,2025-07-31T19:43:24Z,241.413,Steine Sind Steine - Bonus Track,And One,Bodypop


In [106]:
clean_df.to_csv('Extended_History_Clean.csv')